<div style="background-color:#000;"><img src="pqn.png"></img></div>

This code downloads stock and factor returns data, constructs a portfolio, and performs factor analysis. It uses Yahoo Finance for data retrieval and Riskfolio for portfolio optimization. Specifically, it calculates statistical properties of the portfolio and factors, generates factor loadings, and optimizes the portfolio using a factor model. The final output is a pie chart visualizing the optimized portfolio weights. This is useful for investors aiming to optimize their portfolios considering multiple factors.

In [ ]:
import yfinance as yf
import riskfolio as rf
import pandas as pd
pd.options.display.float_format = "{:.4%}".format
import warnings
warnings.filterwarnings("ignore")

Define a list of major technology stocks and factor tickers

In [ ]:
mag_7 = [
    "AMZN",
    "AAPL",
    "NVDA",
    "META",
    "TSLA",
    "MSFT",
    "GOOG",
]

Define a list of factor indices

In [ ]:
factors = ["MTUM", "QUAL", "VLUE", "SIZE", "USMV"]

Define the start and end dates for the data download

In [ ]:
start = "2020-01-01"
end = "2024-07-31"

Download adjusted close price data for the selected stocks and compute daily returns

In [ ]:
port_returns = (
    yf
    .download(
        mag_7, 
        start=start, 
        end=end
    )["Adj Close"]
    .pct_change()
    .dropna()
)

Download adjusted close price data for the selected factors and compute daily returns

In [ ]:
factor_returns = (
    yf
    .download(
        factors, 
        start=start, 
        end=end
    )["Adj Close"]
    .pct_change()
    .dropna()
)

Create a portfolio object and calculate historical mean returns and Ledoit-Wolf shrinkage covariance matrix

In [ ]:
port = rf.Portfolio(returns=port_returns)
port.assets_stats(method_mu="hist", method_cov="ledoit")

Set the minimum acceptable return for the portfolio optimization

In [ ]:
port.lowerret = 0.00056488 * 1.5

Compute the loadings matrix using Principal Component Regression (PCR) with 95% explained variance

In [ ]:
loadings = rf.loadings_matrix(
    X=factor_returns,
    Y=port_returns, 
    feature_selection="PCR",
    n_components=0.95
)

Display the loadings matrix with formatting and background gradient

In [ ]:
loadings.style.format("{:.4f}").background_gradient(cmap='RdYlGn')

Assign factor returns to the portfolio and calculate factor statistics

In [ ]:
port.factors = factor_returns
port.factors_stats(
    method_mu="hist",
    method_cov="ledoit",
    feature_selection="PCR",  # Method to select best model, could be PCR or Stepwise,
    dict_risk=dict(
        n_components=0.95  # 95% of explained variance.
    )
)

Optimize the portfolio using a Factor Model, variance as the risk measure, and Sharpe ratio as the objective

In [ ]:
w = port.optimization(
    model="FM",  # Factor model
    rm="MV",  # Risk measure used, this time will be variance
    obj="Sharpe",  # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    hist=False,  # Use risk factor model for expected returns
)

Plot the optimized portfolio weights as a pie chart

In [ ]:
ax = rf.plot_pie(
    w=w,
    title='Sharpe FM Mean Variance',
    others=0.05,
    nrow=25,
    cmap="tab20"
)

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.